# Importação das bibliotecas

In [60]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from matplotlib import pyplot
from gensim.models import KeyedVectors

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TRT05\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Importação dos dados dos livros

In [61]:
df = pd.read_csv("livros.csv")
df['descricao'] = df['descricao'].astype(str)
df[['titulo', 'autor', 'descricao']]

,titulo,autor,descricao
0,Orçamento sem falhas,Nath Finanças,"Quando o assunto é dinheiro, não dá para achar..."
1,Minha Sombria Vanessa,Kate Elizabeth Russell,Elogiado por Gillian Flynn e considerado um do...
2,Recursão,Blake Crouch,E se um dia memórias vívidas de coisas que nun...
3,"M, o Filho do Século",Antonio Scurati,"O romance M, o Filho do Século conta em trama ..."
4,Oblivion Song: Entre Dois Mundos,Robert Kirkman,Mestre em traçar universos distópicos permeado...
...,...,...,...
11970,Poemas completos de Alberto Caeiro,Fernando Pessoa,"Nos Poemas Completos de Alberto Caeiro (1946),..."
11971,Poesia de Álvaro de Campos,Fernando Pessoa,"Cerebral e retraído, Fernando Pessoa concebeu ..."
11972,Ben-Hur,Lewis Wallace,"O romance épico ""Ben-Hur - Uma História dos Te..."
11973,Folhas de Relva,Walt Whitman,Walt Whitman é considerado pela crítica mundia...


# Pré-processando a descrição

## Removendo números

In [62]:
df['descricao'] = df['descricao'].str.replace('\d+', '')
df['descricao']

C:\Users\TRT05\AppData\Local\Temp\ipykernel_13808\3336066140.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['descricao'] = df['descricao'].str.replace('\d+', '')


0        Quando o assunto é dinheiro, não dá para achar...
1        Elogiado por Gillian Flynn e considerado um do...
2        E se um dia memórias vívidas de coisas que nun...
3        O romance M, o Filho do Século conta em trama ...
4        Mestre em traçar universos distópicos permeado...
                               ...                        
11970    Nos Poemas Completos de Alberto Caeiro (), em ...
11971    Cerebral e retraído, Fernando Pessoa concebeu ...
11972    O romance épico "Ben-Hur - Uma História dos Te...
11973    Walt Whitman é considerado pela crítica mundia...
11974    O Sermão da Montanha é uma das mais belas mens...
Name: descricao, Length: 11975, dtype: object

## Removendo capitalização

In [63]:
df['descricao'] = df['descricao'].str.lower()
df['descricao']

0        quando o assunto é dinheiro, não dá para achar...
1        elogiado por gillian flynn e considerado um do...
2        e se um dia memórias vívidas de coisas que nun...
3        o romance m, o filho do século conta em trama ...
4        mestre em traçar universos distópicos permeado...
                               ...                        
11970    nos poemas completos de alberto caeiro (), em ...
11971    cerebral e retraído, fernando pessoa concebeu ...
11972    o romance épico "ben-hur - uma história dos te...
11973    walt whitman é considerado pela crítica mundia...
11974    o sermão da montanha é uma das mais belas mens...
Name: descricao, Length: 11975, dtype: object

## Removendo pontuação

In [64]:
df['descricao'] = df['descricao'].str.replace('[^\w\s]', '')
df['descricao']

C:\Users\TRT05\AppData\Local\Temp\ipykernel_13808\2231285731.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['descricao'] = df['descricao'].str.replace('[^\w\s]', '')


0        quando o assunto é dinheiro não dá para achar ...
1        elogiado por gillian flynn e considerado um do...
2        e se um dia memórias vívidas de coisas que nun...
3        o romance m o filho do século conta em trama e...
4        mestre em traçar universos distópicos permeado...
                               ...                        
11970    nos poemas completos de alberto caeiro  em ver...
11971    cerebral e retraído fernando pessoa concebeu o...
11972    o romance épico benhur  uma história dos tempo...
11973    walt whitman é considerado pela crítica mundia...
11974    o sermão da montanha é uma das mais belas mens...
Name: descricao, Length: 11975, dtype: object

## Removendo stopwords

In [69]:
stop = stopwords.words('portuguese')

df['descricao'] = df['descricao'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

df['descricao']

0        assunto dinheiro dá achar discurso vai funcion...
1        elogiado gillian flynn considerado grandes liv...
2        dia memórias vívidas coisas nunca aconteceram ...
3        romance m filho século conta trama eletrizante...
4        mestre traçar universos distópicos permeados r...
                               ...                        
11970    poemas completos alberto caeiro versos simples...
11971    cerebral retraído fernando pessoa concebeu pro...
11972    romance épico benhur história tempos cristo pu...
11973    walt whitman considerado crítica mundial maior...
11974    sermão montanha belas mensagens literatura esp...
Name: descricao, Length: 11975, dtype: object

## Lematização

In [80]:
import stanza
nlp_stanza = stanza.Pipeline(lang='pt', processors='tokenize,mwt,pos,lemma')

2022-11-02 09:27:12 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2022-11-02 09:27:13 INFO: Loading these models for language: pt (Portuguese):
| Processor | Package |
-----------------------
| tokenize  | bosque  |
| mwt       | bosque  |
| pos       | bosque  |
| lemma     | bosque  |

2022-11-02 09:27:13 INFO: Use device: cpu
2022-11-02 09:27:13 INFO: Loading: tokenize
2022-11-02 09:27:13 INFO: Loading: mwt
2022-11-02 09:27:13 INFO: Loading: pos
2022-11-02 09:27:14 INFO: Loading: lemma
2022-11-02 09:27:14 INFO: Done loading processors!


In [81]:
nlp_stanza("O rato roeu a roupa do rei de roma.")

[
  [
    {
      "id": 1,
      "text": "O",
      "lemma": "o",
      "upos": "DET",
      "feats": "Definite=Def|Gender=Masc|Number=Sing|PronType=Art",
      "start_char": 0,
      "end_char": 1
    },
    {
      "id": 2,
      "text": "rato",
      "lemma": "rato",
      "upos": "NOUN",
      "feats": "Gender=Masc|Number=Sing",
      "start_char": 2,
      "end_char": 6
    },
    {
      "id": 3,
      "text": "roeu",
      "lemma": "roer",
      "upos": "VERB",
      "feats": "Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin",
      "start_char": 7,
      "end_char": 11
    },
    {
      "id": 4,
      "text": "a",
      "lemma": "o",
      "upos": "DET",
      "feats": "Definite=Def|Gender=Fem|Number=Sing|PronType=Art",
      "start_char": 12,
      "end_char": 13
    },
    {
      "id": 5,
      "text": "roupa",
      "lemma": "roupa",
      "upos": "NOUN",
      "feats": "Gender=Fem|Number=Sing",
      "start_char": 14,
      "end_char": 19
    },
    {
      "id": [
 